# Выбор локации для скважины

Нужно решить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. 

## Загрузка и подготовка данных

Импортируем необходимые библиотеки и загрузим данные.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [2]:
data1 = pd.read_csv('/datasets/geo_data_0.csv')
data2 = pd.read_csv('/datasets/geo_data_1.csv')
data3 = pd.read_csv('/datasets/geo_data_2.csv')

Посмотрим на предоставленные данные.  
- id — уникальный идентификатор скважины;  
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);  
- product — объём запасов в скважине (тыс. баррелей)  

In [3]:
print(data1.info())
data1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
print(data2.info())
data2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [5]:
print(data3.info())
data3.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Проверим данные на наличие дубликатов, пропусков и аномальных значений.

In [6]:
data1.duplicated().sum()
data1.isna().sum()
data1['f0'].unique()
data1['f1'].unique()
data1['f2'].unique()
data1['product'].unique()

data2.duplicated().sum()
data2.isna().sum()
data2['f0'].unique()
data2['f1'].unique()
data2['f2'].unique()
data2['product'].unique()

data3.duplicated().sum()
data3.isna().sum()
data3['f0'].unique()
data3['f1'].unique()
data3['f2'].unique()
data3['product'].unique()

array([ 27.75867323,  56.06969663,  62.87191005, ..., 157.08007975,
        51.79525299, 102.77576749])

Дубликатов, пропущенных и аномальных значений нет, данные готовы к работе.

## Обучение и проверка модели

Выделим признаки и целевой признак в данных. Разделим данные на тренировочную и валидационную выборки. Масштабируем признаки.

In [7]:
data1_features = data1[['f0', 'f1', 'f2']]
data1_target = data1['product']
data2_features = data2[['f0', 'f1', 'f2']]
data2_target = data2['product']
data3_features = data3[['f0', 'f1', 'f2']]
data3_target = data3['product']

features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(data1_features, data1_target, test_size=0.25, random_state=12345)
print(len(features_train_1), len(features_valid_1))

features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(data2_features, data2_target, test_size=0.25, random_state=12345)
print(len(features_train_2), len(features_valid_2))

features_train_3, features_valid_3, target_train_3, target_valid_3 = train_test_split(data3_features, data3_target, test_size=0.25, random_state=12345)
print(len(features_train_3), len(features_valid_3))

numeric = ['f0', 'f1', 'f2']
scaler = StandardScaler()

scaler.fit(features_train_1[numeric])
features_train_1[numeric] = scaler.transform(features_train_1[numeric])
features_valid_1[numeric] = scaler.transform(features_valid_1[numeric])

scaler.fit(features_train_2[numeric])
features_train_2[numeric] = scaler.transform(features_train_2[numeric])
features_valid_2[numeric] = scaler.transform(features_valid_2[numeric])

scaler.fit(features_train_3[numeric])
features_train_3[numeric] = scaler.transform(features_train_3[numeric])
features_valid_3[numeric] = scaler.transform(features_valid_3[numeric])

75000 25000
75000 25000
75000 25000


Построим модель линейной регрессии для каждого региогна, определим ее качество, используя корень от среднеквадратичной ошибки, и посчитаем средний запас предсказанного сырья в каждом регионе.

In [22]:
def model(features_train, target_train, features_valid, target_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    mse = mean_squared_error(target_valid, predicted_valid)
    rmse = mse ** 0.5
    print('Средний запас предсказанного сырья в регионе:', predicted_valid.mean())
    print('RMSE модели региона:', rmse)  
    return predicted_valid

In [23]:
model(features_train_1, target_train_1, features_valid_1, target_valid_1)

Средний запас предсказанного сырья в регионе: 92.59256778438035
RMSE модели региона: 37.5794217150813


array([ 95.89495185,  77.57258261,  77.89263965, ...,  61.50983303,
       118.18039721, 118.16939229])

In [10]:
model = LinearRegression()
model.fit(features_train_1, target_train_1)
predicted_valid_1 = model.predict(features_valid_1)
mse1 = mean_squared_error(target_valid_1, predicted_valid_1)
rmse1 = mse1 ** 0.5

print('Средний запас предсказанного сырья в регионе 1:', predicted_valid_1.mean())
print('RMSE модели региона 1:', rmse1)

Средний запас предсказанного сырья в регионе 1: 92.59256778438035
RMSE модели региона 1: 37.5794217150813


In [11]:
model = LinearRegression()
model.fit(features_train_2, target_train_2)
predicted_valid_2 = model.predict(features_valid_2)
mse2 = mean_squared_error(target_valid_2, predicted_valid_2)
rmse2 = mse2 ** 0.5

print('Средний запас предсказанного сырья в регионе 2:', predicted_valid_2.mean())
print('RMSE модели региона 2:', rmse2)

Средний запас предсказанного сырья в регионе 2: 68.728546895446
RMSE модели региона 2: 0.893099286775617


In [12]:
model = LinearRegression()
model.fit(features_train_3, target_train_3)
predicted_valid_3 = model.predict(features_valid_3)
mse3 = mean_squared_error(target_valid_3, predicted_valid_3)
rmse3 = mse3 ** 0.5

print('Средний запас предсказанного сырья в регионе 1:', predicted_valid_3.mean())
print('RMSE модели региона 1:', rmse3)

Средний запас предсказанного сырья в регионе 1: 94.96504596800489
RMSE модели региона 1: 40.02970873393434


**По результатам работы моделей, можно сделать вывод о том, что выбор стоит делать между 1 и 3 регионами. 2 регион показал наименьший запас предсказанного сырья и плохое качество работы модели. Показатели 1 и 3 регионов практически не отличаются.**

## Подготовка к расчёту прибыли

Переведем предсказания в необходимый формат.

In [13]:
predicted_valid_1 = pd.Series(predicted_valid_1, index=target_valid_1.index)
predicted_valid_2 = pd.Series(predicted_valid_2, index=target_valid_2.index)
predicted_valid_3 = pd.Series(predicted_valid_3, index=target_valid_3.index)

Подготовим переменные, необходимые для расчета прибыли.

In [14]:
BUDGET = 10000000000
INCOME_FOR_ONE = 450000

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины.

In [15]:
BUDGET / 200 / INCOME_FOR_ONE

111.11111111111111

111 тыс. барелей - необходимый объем сырья в скважине для того, чтобы разработка стала безубыточной. Сравним этот показатель со средним запасом в каждом регионе.

In [16]:
print(data1['product'].mean())
print(data2['product'].mean())
print(data3['product'].mean())

92.50000000000001
68.82500000000002
95.00000000000004


По среднему показателю ни один регион не дотягивает до необходимого значения. Самый маленький показатель в регионе 2.

## Расчёт прибыли и рисков 

Напишем функцию для подсчета прибыли.

In [17]:
def profit(target, predicted, count):
    pred_sorted = predicted.sort_values(ascending=False)
    selected = target[pred_sorted.index][:count]
    profit = selected.sum() * INCOME_FOR_ONE - BUDGET
    return profit
    print(pred_sorted.head())
    print(selected.head())

Напишем функцию для нахождения распределения прибыли и риска убытков техникой bootstrap.

In [18]:
def bootstrap(target, predictions):
    state = np.random.RandomState(12345)
    values = []
    loss = 0
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        pred_subsample = predictions[target_subsample.index]
        values.append(profit(target_subsample, pred_subsample, 200))
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    risk = (values < 0). mean() * 100
    print('Средняя прибыль:', values.mean())
    print('95% доверительный интервал:', lower,upper)
    print('Риск убытков:', risk, '%')

Оценим ситуацию в регионе 1.

In [19]:
bootstrap(target_valid_1, predicted_valid_1)

Средняя прибыль: 425938526.91059244
95% доверительный интервал: -102090094.83793654 947976353.3583689
Риск убытков: 6.0 %


Оценим ситуацию в регионе 2.

In [20]:
bootstrap(target_valid_2, predicted_valid_2)

Средняя прибыль: 515222773.4432899
95% доверительный интервал: 68873225.37050176 931547591.2570494
Риск убытков: 1.0 %


Оценим ситуацию в регионе 3.

In [21]:
bootstrap(target_valid_3, predicted_valid_3)

Средняя прибыль: 435008362.7827556
95% доверительный интервал: -128880547.32978901 969706954.1802661
Риск убытков: 6.4 %


**Финальный вывод**  
По итогам работы были обработаны данные и предсказаны запасы сырья. Была оценена средняя прибыль каждого из регионов, распределение прибыли, риск убытков. Наилучший показатель средней прибыли (515222773) и наименьшая вероятность убытков (1%) была показана в регионе 2. Таким образом, регион 2 является наиболее удачным выбором для разработки новых скважин.